# A/B Testing Simulation

In this notebook, a user has a hidden preference within a single query. We use this to explore A/B testing. 

Of course, this problem needs to be multiplied over millions of queries!

1. The last judgments from chapter 11
2. Fully train a model w/ two feature sets (turn ch 11 auto LTR notebook into function) 
3. Simulate user interaction w/ rankings

In [11]:
import numpy as np
import pandas as pd
import random; random.seed(0)
import glob

import requests
import sys
sys.path.append('..')
from ltr.client.solr_client import SolrClient

client = SolrClient(host='http://aips-solr:8983/solr')

In [12]:
def all_sessions():
    sessions = pd.concat([pd.read_csv(f, compression='gzip')
                          for f in glob.glob('retrotech/sessions/*_sessions.gz')])
    return sessions.rename(columns={'clicked_doc_id': 'doc_id'})
    
sessions = all_sessions()
sessions

,sess_id,query,rank,doc_id,clicked
0,50002,blue ray,0.0,600603141003,True
1,50002,blue ray,1.0,827396513927,False
2,50002,blue ray,2.0,24543672067,False
3,50002,blue ray,3.0,719192580374,False
4,50002,blue ray,4.0,885170033412,True
...,...,...,...,...,...
74995,5001,transformers dark of the moon,10.0,47875841369,False
74996,5001,transformers dark of the moon,11.0,97363560449,False
74997,5001,transformers dark of the moon,12.0,93624956037,False
74998,5001,transformers dark of the moon,13.0,97363532149,False


In [13]:
sessions['query'].unique()

array(['blue ray', 'bluray', 'dryer', 'headphones', 'ipad', 'iphone',
       'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
       'transformers dark of the moon'], dtype=object)

In [14]:
new_sessions = sessions[sessions['query'] == 'macbook'].copy() 

In [15]:
random.seed(0)

# Make two queries identical, except for the query
# TODO? Randomly flip some of the clicked bools, but this might make it non deterministic
def copy_query_sessions(sessions, src_query, dest_query):
    new_sessions = sessions[sessions['query'] == src_query].copy()  
    new_sessions['draw'] = np.random.rand(len(new_sessions), 1)
    # unclick some in the new query for a bit of noise
    new_sessions[new_sessions['clicked'] & (new_sessions['draw'] < 0.04)]['clicked'] = False
    new_sessions['query'] = dest_query
    return pd.concat([sessions, new_sessions.drop('draw', axis=1)])

sessions = copy_query_sessions(sessions, 'transformers dark of the moon', 'transformers dark of moon')
sessions = copy_query_sessions(sessions, 'transformers dark of the moon', 'dark of moon')
sessions = copy_query_sessions(sessions, 'transformers dark of the moon', 'dark of the moon')
sessions = copy_query_sessions(sessions, 'headphones', 'head phones')
sessions = copy_query_sessions(sessions, 'lcd tv', 'lcd television')
sessions = copy_query_sessions(sessions, 'lcd tv', 'television, lcd')
sessions = copy_query_sessions(sessions, 'macbook', 'apple laptop')
sessions = copy_query_sessions(sessions, 'iphone', 'apple iphone')
sessions = copy_query_sessions(sessions, 'kindle', 'amazon kindle')
sessions = copy_query_sessions(sessions, 'kindle', 'amazon ereader')
sessions = copy_query_sessions(sessions, 'blue ray', 'blueray')





sessions

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,sess_id,query,rank,doc_id,clicked
0,50002,blue ray,0.0,600603141003,True
1,50002,blue ray,1.0,827396513927,False
2,50002,blue ray,2.0,24543672067,False
3,50002,blue ray,3.0,719192580374,False
4,50002,blue ray,4.0,885170033412,True
...,...,...,...,...,...
149995,55001,blueray,25.0,22265004517,False
149996,55001,blueray,26.0,885170038875,False
149997,55001,blueray,27.0,786936817232,False
149998,55001,blueray,28.0,600603132872,False


In [16]:
sessions['query'].unique()

array(['blue ray', 'bluray', 'dryer', 'headphones', 'ipad', 'iphone',
       'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
       'transformers dark of the moon', 'transformers dark of moon',
       'dark of moon', 'dark of the moon', 'head phones',
       'lcd television', 'television, lcd', 'apple laptop',
       'apple iphone', 'amazon kindle', 'amazon ereader', 'blueray'],
      dtype=object)

## Inject bias for transformers / transformers dvd

This simulates biased sessions in the data, as if the user never actually sees (and hence never clicks) their actual desired item. If the users desired results are shown, those results get a higher probability of click. Otherwise there is a lower probability of clicks.

In [17]:
next_sess_id = sessions['sess_id'].max()

# For some reason, the sessions only capture examines on the 'dubbed' transformers movies
# ie the Japanese shows brought to an English-speaking market. But we'll see this is not what the 
# user wants (ie presentation bias). These are 'meh' mildly interesting. There are also many many
# completely irrelevant movies.

# What the user wants, but never visible! Never gets clicked!
# These are the widescreen transformers dvds of the hollywood movies
desired_movies = ["97360724240", "97360722345", "97368920347"] 

# Bunch of random merchandise
irrelevant_transformers_products = ["708056579739", "93624995012", "47875819733", "47875839090", "708056579746",
                                     "47875332911", "47875842328", "879862003524", "879862003517", "93624974918",
                                     ] 

# Other transformer movies
meh_transformers_movies = ["97363455349", "97361312743", "97361372389", "97361312804", "97363532149", "97363560449"]

displayed_transformer_products = meh_transformers_movies + irrelevant_transformers_products

new_sessions = []
for i in range(0,5000):
    random.shuffle(displayed_transformer_products)

    # shuffle each session
    for rank, upc in enumerate(displayed_transformer_products):
        clicked = False
        draw = random.random()

        if upc in meh_transformers_movies:
            if draw < 0.13:
                clicked = True
        elif upc in irrelevant_transformers_products:
            if draw < 0.005:
                clicked = True
        elif upc in desired_transformers_movies:
            if draw < 0.65:
                clicked = True

        new_sessions.append({'sess_id': next_sess_id + i, 
                             'query': 'transformers dvd', 
                             'rank': rank,
                             'clicked': clicked,
                             'doc_id': upc
                             })


sessions = sessions.append(new_sessions)

## Chapter 11 In One Function

Wrapping up Chapter 11 in a single function `sessions_to_sdbn`

In [18]:
def sessions_to_sdbn(sessions, prior_weight=10, prior_grade=0.2) -> pd.DataFrame:
    """ Compute SDBN of the provided query as a dataframe.
        Where we left off at end of 'overcoming confidence bias' 
        """
    all_sdbn = pd.DataFrame()
    for query in sessions['query'].unique():
        sdbn_sessions = sessions[sessions['query'] == query].copy().set_index('sess_id')

        last_click_per_session = sdbn_sessions.groupby(['clicked', 'sess_id'])['rank'].max()[True]

        sdbn_sessions['last_click_rank'] = last_click_per_session
        sdbn_sessions['examined'] = sdbn_sessions['rank'] <= sdbn_sessions['last_click_rank']

        sdbn = sdbn_sessions[sdbn_sessions['examined']].groupby('doc_id')[['clicked', 'examined']].sum()
        sdbn['grade'] = sdbn['clicked'] / sdbn['examined']
        sdbn['query'] = query

        sdbn = sdbn.sort_values('grade', ascending=False)

        sdbn['prior_a'] = prior_grade*prior_weight
        sdbn['prior_b'] = (1-prior_grade)*prior_weight

        sdbn['posterior_a'] = sdbn['prior_a'] +  sdbn['clicked']
        sdbn['posterior_b'] = sdbn['prior_b'] + (sdbn['examined'] - sdbn['clicked'])

        sdbn['beta_grade'] = sdbn['posterior_a'] / (sdbn['posterior_a'] + sdbn['posterior_b'])

        sdbn.sort_values('beta_grade', ascending=False)
        all_sdbn = all_sdbn.append(sdbn)
    return all_sdbn[['query', 'clicked', 'examined', 'grade', 'beta_grade']].reset_index().set_index(['query', 'doc_id'])

queries = ['dryer', 'bluray', 'blue ray', 'headphones', 'ipad', 'iphone',
           'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
           'transformers dark of the moon']



In [19]:
sessions

,sess_id,query,rank,doc_id,clicked
0,50002,blue ray,0.0,600603141003,True
1,50002,blue ray,1.0,827396513927,False
2,50002,blue ray,2.0,24543672067,False
3,50002,blue ray,3.0,719192580374,False
4,50002,blue ray,4.0,885170033412,True
...,...,...,...,...,...
79995,65000,transformers dvd,11.0,47875842328,False
79996,65000,transformers dvd,12.0,879862003517,False
79997,65000,transformers dvd,13.0,97361372389,False
79998,65000,transformers dvd,14.0,93624995012,False


## Listing 12.1 Use Convert Raw Sessions to SDBN

In [20]:
sdbn = sessions_to_sdbn(sessions,
                        prior_weight=10,
                        prior_grade=0.2)
sdbn

clicked  examined     grade  beta_grade
query            doc_id                                               
blue ray         27242815414      42.0      42.0  1.000000    0.846154
                 600603132872     46.0      88.0  0.522727    0.489796
                 827396513927   1304.0    3381.0  0.385685    0.385137
                 600603141003    978.0    2620.0  0.373282    0.372624
                 885170033412    568.0    2184.0  0.260073    0.259799
...                                ...       ...       ...         ...
transformers dvd 47875819733      24.0    1679.0  0.014294    0.015394
                 708056579739     23.0    1659.0  0.013864    0.014979
                 879862003524     23.0    1685.0  0.013650    0.014749
                 93624974918      19.0    1653.0  0.011494    0.012628
                 47875839090      16.0    1669.0  0.009587    0.010721

[649 rows x 4 columns]

## Chapter 10 Functions (omitted from book)

All of the following are support functions for the chapter:

1. Convert the sdbn dataframe into individual `Judgment` objects needed for training the model from chapter 10
2. Pairwise transformation of the data
3. Normalization of the data
4. Training the model
5. Uploading the model to Solr

All of these steps are covered in Chapter 10.

In [21]:
import requests
import numpy as np
from ltr.judgments import judgments_from_file, judgments_to_nparray
from sklearn import svm
import json
import math
from itertools import groupby
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby
from ltr import download
from ltr.judgments import judgments_writer

from ltr.judgments import Judgment

def sdbn_to_judgments(sdbn):
    """Turn pandas dataframe into ltr judgments objects."""
    judgments = []
    queries = {}
    next_qid = 0
    for row_dict in sdbn.reset_index().to_dict(orient="records"):
        # Round grade to 10ths, Map 0.3 -> 3, etc
        grade = round(row_dict['beta_grade'], 1) * 10
        qid = -1
        if row_dict['query'] in queries:
            qid = queries[row_dict['query']]
        else:
            queries[row_dict['query']] = next_qid
            qid = next_qid
            next_qid += 1
        assert qid != -1
        
        judgments.append(Judgment(doc_id=row_dict['doc_id'],
                                  keywords=row_dict['query'],
                                  qid=qid,
                                  grade=int(grade))
                        )
    return judgments


sdbn_to_judgments(sdbn)


def write_judgments(judgments, dest='retrotech_judgments.txt'):
    with judgments_writer(open(dest, 'wt')) as writer:
        for judgment in judgments:
            writer.write(judgment)
            
write_judgments(sdbn_to_judgments(sdbn))
!cat retrotech_judgments.txt


def normalize_features(logged_judgments):
    all_features = []
    means = [0] * len(logged_judgments[0].features)
    for judgment in logged_judgments:
        for idx, f in enumerate(judgment.features):
            means[idx] += f
        all_features.append(judgment.features)
    
    for i in range(len(means)):
        means[i] /= len(logged_judgments)
      
    std_devs = [0.0] * len(logged_judgments[0].features)
    for judgment in logged_judgments:
        for idx, f in enumerate(judgment.features):
            std_devs[idx] += (f - means[idx])**2
            
    for i in range(len(std_devs)):
        std_devs[i] /= len(logged_judgments)
        std_devs[i] = math.sqrt(std_devs[i])
        
    # Normalize!
    normed_judgments = []
    for judgment in logged_judgments:
        normed_features = [0.0] * len(judgment.features)
        for idx, f in enumerate(judgment.features):
            normed = (f - means[idx]) / std_devs[idx]
            normed_features[idx] = normed
        normed_judgment=Judgment(qid=judgment.qid,
                                 keywords=judgment.keywords,
                                 doc_id=judgment.doc_id,
                                 grade=judgment.grade,
                                 features=normed_features)
        normed_judgment.old_features=judgment.features
        normed_judgments.append(normed_judgment)

    return means, std_devs, normed_judgments


def pairwise_transform(normed_judgments, weigh_difference = True):
        
    predictor_deltas = []
    feature_deltas = []
    
    # For each query's judgments
    for qid, query_judgments in groupby(normed_judgments, key=lambda j: j.qid):

        # Annoying issue consuming python iterators, we ensure we have two
        # full copies of each query's judgments
        query_judgments_copy_1 = list(query_judgments) 
        query_judgments_copy_2 = list(query_judgments_copy_1)

        # Examine every judgment combo for this query, 
        # if they're different, store the pairwise difference:
        # +1 if judgment1 more relevant
        # -1 if judgment2 more relevant
        for judgment1 in query_judgments_copy_1:
            for judgment2 in query_judgments_copy_2:
                
                j1_features=np.array(judgment1.features)
                j2_features=np.array(judgment2.features)
                
                if judgment1.grade > judgment2.grade:
                    diff = judgment1.grade - judgment2.grade if weigh_difference else 1.0
                    predictor_deltas.append(+1)
                    feature_deltas.append(diff * (j1_features-j2_features))
                elif judgment1.grade < judgment2.grade:
                    diff = judgment2.grade - judgment1.grade if weigh_difference else 1.0
                    predictor_deltas.append(-1)
                    feature_deltas.append(diff * (j1_features-j2_features))

    # For training purposes, we return these as numpy arrays
    return np.array(feature_deltas), np.array(predictor_deltas)

def upload_model(model, means, std_devs, feature_set):

    linear_model = {
      "store": "test",
      "class": "org.apache.solr.ltr.model.LinearModel",
      "name": "test_model",
      "features": [
      ],
      "params": {
          "weights": {
          }
      }
    }

    ftr_model = {}
    ftr_names = [ftr['name'] for ftr in feature_set]
    for idx, ftr_name in enumerate(ftr_names):
        config = {
            "name": ftr_name,
            "norm": {
                "class": "org.apache.solr.ltr.norm.StandardNormalizer",
                "params": {
                    "avg": str(means[idx]),
                    "std": str(std_devs[idx])
                }
            }
        }
        linear_model['features'].append(config)
        linear_model['params']['weights'][ftr_name] =  model.coef_[0][idx] 

    print("PUT http://aips-solr:8983/solr/products/schema/model-store")
    print(json.dumps(linear_model, indent=2))

    # Delete old model
    resp = requests.delete('http://aips-solr:8983/solr/products/schema/model-store/test_model')


    # Upload the model
    resp = requests.put('http://aips-solr:8983/solr/products/schema/model-store', json=linear_model)
    resp.text
    
    
## TODO - can't easily to test/train split on these few queries
##   make more queries?

def ranksvm_ltr(sdbn, feature_set):
    """Train a RankSVM model via Solr, store in Solr."""
    judgments = sdbn_to_judgments(sdbn)
    judgments_path = 'retrotech_judgments.txt'
    write_judgments(judgments, judgments_path)
    
    # For more on this code, review Chapter 10
    requests.delete('http://aips-solr:8983/solr/products/schema/feature-store/test')
    
    resp = requests.put('http://aips-solr:8983/solr/products/schema/feature-store',
                    json=feature_set)

    ftr_logger=FeatureLogger(client, index='products', feature_set='test', id_field='upc')

    with judgments_open(judgments_path) as judgment_list:
        for qid, query_judgments in groupby(judgments, key=lambda j: j.qid):
            ftr_logger.log_for_qid(judgments=query_judgments, 
                                   qid=qid,
                                   keywords=judgment_list.keywords(qid))

    logged_judgments = ftr_logger.logged
    means, std_devs, normed_judgments = normalize_features(logged_judgments)
    feature_deltas, predictor_deltas = pairwise_transform(normed_judgments)

    model = svm.LinearSVC(max_iter=10000, verbose=1)
    model.fit(feature_deltas, predictor_deltas)    
    upload_model(model, means, std_devs, feature_set)


# qid:0: blue ray*1
# qid:1: bluray*1
# qid:2: dryer*1
# qid:3: headphones*1
# qid:4: ipad*1
# qid:5: iphone*1
# qid:6: kindle*1
# qid:7: lcd tv*1
# qid:8: macbook*1
# qid:9: nook*1
# qid:10: star trek*1
# qid:11: star wars*1
# qid:12: transformers dark of the moon*1
# qid:13: transformers dark of moon*1
# qid:14: dark of moon*1
# qid:15: dark of the moon*1
# qid:16: head phones*1
# qid:17: lcd television*1
# qid:18: television, lcd*1
# qid:19: apple laptop*1
# qid:20: apple iphone*1
# qid:21: amazon kindle*1
# qid:22: amazon ereader*1
# qid:23: blueray*1
# qid:24: transformers dvd*1

8	qid:0	 # 27242815414	blue ray
5	qid:0	 # 600603132872	blue ray
4	qid:0	 # 827396513927	blue ray
4	qid:0	 # 600603141003	blue ray
3	qid:0	 # 885170033412	blue ray
3	qid:0	 # 883929140855	blue ray
2	qid:0	 # 24543672067	blue ray
2	qid:0	 # 813774010904	blue ray
2	qid:0	 # 36725617605	blue ray
2	qid:0	 # 786936817232	blue ray
2	qid:0	 # 36725608443	blue ray
2	qid:0	 # 719192580374	blue ray
2	qid:0	 # 25192

## Also Chapter 10 - Perform a test / train split on the SDBN data

This function is broken out from the model training. It lets us train a model on one set of data (reusing the chapter 10 training code), reserving test queries for evaluation.

In [22]:
from math import floor

def test_train_split(sdbn, train):
    """Split queries in sdbn into train / test split with `train` proportion going to training set."""
    queries = sdbn.index.get_level_values('query').unique().copy().tolist()
    random.shuffle(queries)
    num_queries = len(queries)
    split_point = floor(num_queries * train)
    
    train_queries = queries[:split_point]
    test_queries = queries[split_point:]
    return sdbn.loc[train_queries, :], sdbn.loc[test_queries]


## Chapter 10 - Search Code

Also from Chapter 10, a simple function to search using the LTR model and return a list of search results.

In [23]:
def search(query, at=10):
    """ Search using test_model LTR model (see rq to and qf params below). """
    request = {
            "fields": ["upc", "name", "manufacturer", "score"],
            "limit": at,
            "params": {
              "rq": "{!ltr reRankDocs=60000 reRankWeight=10.0 model=test_model efi.keywords=\"" + query + "\"}",
              "qf": "name upc manufacturer shortDescription longDescription",
              "defType": "edismax",
              "q": query
            }
        }

    search_results = requests.post('http://aips-solr:8983/solr/products/select', 
                                   json=request).json()["response"]["docs"]

    for rank, result in enumerate(search_results):
        result['rank'] = rank
        
    return search_results

## Evaluate the model on the test set

This function computes the model's performance on a set of test queries.

In [24]:
def eval_model(test, at=10):
    queries = test.index.get_level_values('query').unique()
    collection = "products"
    
    query_results = {}
    
    for query in queries:
        search_results = search(query, at=at)
      

        results = pd.DataFrame(search_results).reset_index()
        judgments = sdbn.loc[query, :].copy().reset_index()
        judgments['doc_id'] = judgments['doc_id'].astype(str)
        if len(results) == 0:
            print(f"No Results for {query}")
            query_results[query] = 0
        else:
            graded_results = results.merge(judgments, left_on='upc', right_on='doc_id', how='left')
            graded_results[['clicked', 'examined', 'grade', 'beta_grade']] = graded_results[['clicked', 'examined', 'grade', 'beta_grade']].fillna(0)
            grade_results = graded_results.drop('doc_id', axis=1)

            query_results[query] = (graded_results['beta_grade'].sum() / at)
    return query_results

In [58]:
def live_user_query(query, desired, interesting, uninteresting, 
                    desired_prob=0.5, interesting_prob=0.1, uninteresting_prob=0.01):
    """Live user for 'query' where purchase probability depends on if 
       products upc is in one of three sets.
       
       Users purchase a single product per session.       
       
       """   
    search_results = search(query, at=10)

    results = pd.DataFrame(search_results).reset_index()
    for doc in results.to_dict(orient="records"):
        clicked = False
        draw = random.random()
        
        upc = doc['upc']
        print(doc['name'])

        if upc in desired:
            print(f"Desired {upc}")
            if draw < desired_prob:
                return True
            print("No purchase")
        elif upc in interesting:
            print(f"Interesting {upc}")
            if draw < interesting_prob:
                return True
        elif upc in uninteresting:
            print(f"Uninteresting {upc}")
            if draw < uninteresting_prob:
                return True
        else:
            print(f"Completely unrelated {upc}")
    
    return False

live_user_query(query='transformers dvd', 
                desired=desired_movies,
                interesting=meh_transformers_movies,
                uninteresting=irrelevant_transformers_products)

Transformers - DVD
Completely unrelated 32429037763
Richard Simmons: Boogie Down the Pounds - DVD
Completely unrelated 610583357998
George Michael: Live in London - DVD
Completely unrelated 886976044299
Moon - Widescreen Dubbed Subtitle AC3 - DVD
Completely unrelated 43396315259
Kids - DVD
Completely unrelated 31398751823
Make It or Break It, Vol. 1 [2 Discs] - Widescreen Subtitle AC3 - DVD
Completely unrelated 786936800432
Tactical Unit: The Code - Widescreen Subtitle AC3 Dolby - DVD
Completely unrelated 112120637018
Little Ashes - AC3 - DVD
Completely unrelated 741952675495
Facing Ali - Widescreen Dubbed Subtitle AC3 - DVD
Completely unrelated 31398115939
Change Of Heart (1998) - DVD
Completely unrelated 30306771090


False

## Listing 12.2 - model training

In [43]:

feature_set = [
    {
      "name" : "long_description_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "longDescription:(${keywords})"
      }
    },
    {
      "name" : "short_description_constant",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "shortDescription:(${keywords})^=1"
      }
    }
]

train, test = test_train_split(sdbn, train=0.8)
ranksvm_ltr(train, feature_set) # chapter 10: train the model -> the 'LTR engine'
eval_model(test)

Searching products [Status: 200]
Searching products [Status: 200]
Missing doc 600603124570
Searching products [Status: 200]
Missing doc 600603132827
Searching products [Status: 200]
Missing doc 600603140631
Missing doc 600603125065
Missing doc 600603132827
Missing doc 600603133237
Searching products [Status: 200]
Searching products [Status: 200]
Missing doc 600603139758
Searching products [Status: 200]
Missing doc 600603135101
Searching products [Status: 200]
Searching products [Status: 200]
Searching products [Status: 200]
Searching products [Status: 200]
Searching products [Status: 200]
Searching products [Status: 200]
Searching products [Status: 200]
Missing doc 600603139758
Searching products [Status: 200]
Missing doc 600603123061
Searching products [Status: 200]
Missing doc 600603140631
Missing doc 600603125065
Missing doc 600603132827
Missing doc 600603133237
Searching products [Status: 200]
Searching products [Status: 200]
Searching products [Status: 200]
Missing doc 60060313287

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



{'blue ray': 0.0,
 'bluray': 0,
 'macbook': 0.3099797825224855,
 'head phones': 0.0,
 'lcd television': 0.01739651043448512}

In [52]:
## Listing 12.3 - model training

In [ ]:
random.seed(0)

feature_set_better = [
    {
      "name" : "name_phrase_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "name:\"(${keywords})\""
      }
    },
    {
      "name" : "name_constant",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "name:(${keywords})^=1"
      }
    },
    {
      "name" : "name_bm25",
      "store": "test",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : { #q=title:({$keywords})
        "q" : "name:(${keywords})^=1"
      }
    }
]

sdbn = sessions_to_sdbn(sessions) # chapter 11: generate training data

train, test = test_train_split(sdbn, train=0.8)
ranksvm_ltr(train, feature_set_better) # chapter 10: train the model -> the 'LTR engine'
eval2 = eval_model(test)

eval2

In [62]:
train_cg_1 = []
train_cg_2 = []
for i in range(0,25):
    train, test = test_train_split(sdbn, train=0.8)
    
    ranksvm_ltr(train, feature_set_meh) # chapter 10: train the model -> the 'LTR engine'
    eval1 = eval_model(test)
    
    ranksvm_ltr(train, feature_set_better) # chapter 10: train the model -> the 'LTR engine'
    eval2 = eval_model(test)
    
    train_cg_1.append(sum(eval1.values()) / len(eval1))
    train_cg_2.append(sum(eval2.values()) / len(eval2))
    
train_cg_1, train_cg_2, sum(train_cg_1), sum(train_cg_2)

NameError: name 'feature_set_meh' is not defined

In [274]:
sum(train_cg_1) / 25, sum(train_cg_2) / 25

(0.045335276014046196, 0.06695738942332295)

In [273]:
np.std(train_cg_1), np.std(train_cg_2)

(0.026037216521422997, 0.03632680499698539)

In [260]:
# Cumulative Gain
print(eval2.keys())
sum(eval2.values()) / len(eval2)

dict_keys(['blue ray', 'headphones', 'kindle', 'apple laptop', 'apple iphone'])


0.0674481302750636

In [60]:
from IPython.core.display import display,HTML
from aips import render_search_results

query = "transformers dvd"

collection = "products"
request = {
    "fields": ["upc", "name", "manufacturer", "score"],
    "limit": 5,
    "params": {
      "rq": "{!ltr reRankDocs=60000 reRankWeight=200.0 model=test_model efi.keywords=\"" + query + "\"}",
      "qf": "name upc manufacturer shortDescription longDescription",
      "defType": "edismax",
      "q": query
    }
}

search_results = requests.post('http://aips-solr:8983/solr/products/select', json=request).json()["response"]["docs"]
display(HTML(render_search_results(query, search_results)))

In [119]:
sessions['query'].unique()

array(['blue ray', 'bluray', 'dryer', 'headphones', 'ipad', 'iphone',
       'kindle', 'lcd tv', 'macbook', 'nook', 'star trek', 'star wars',
       'transformers dark of the moon', 'transformers dark of moon',
       'dark of moon', 'dark of the moon', 'head phones',
       'lcd television', 'television, lcd', 'apple laptop',
       'apple iphone', 'amazon kindle', 'amazon ereader', 'blueray',
       'transformers dvd'], dtype=object)

In [ ]:
#1. Just use the queries we have to do a test/train split
#2. Simulate the user for A/B test

In [54]:
sessions[sessions['query'] == 'headphones']

,sess_id,query,rank,doc_id,clicked
0,30002,headphones,0.0,803238004525,True
1,30002,headphones,1.0,615104173552,False
2,30002,headphones,2.0,848447000135,False
3,30002,headphones,3.0,27242807785,False
4,30002,headphones,4.0,878615035287,False
...,...,...,...,...,...
149995,35001,headphones,25.0,27242798236,False
149996,35001,headphones,26.0,709483027855,False
149997,35001,headphones,27.0,46838046100,False
149998,35001,headphones,28.0,27242799127,False
